<h1 style="color:orange">
    <a name="top"></a> BQFactor Tutorial
</h1>

<h2 style="color:orange">
    <a name="create context"></a>Create context
</h2>

```python
import bql

bq = bql.Service(preferences={'currencyCheck': 'when_available'})
context = bqfactor.create_analysis_context(universe, backtest_dates, bq, currency='HKD')
```

<h2 style="color:orange">
    <a name="construct factors"></a>Construct factors
</h2>

```python
earnings_0Y = bq.data.net_income(fpt='A',fpo='0',currency='HKD')
earnings_est = bq.data.net_income(fpt='A',fpo='1',currency='HKD')
equity = bq.data.TOT_COMMON_EQY(currency='HKD')   # latest total equity
roe = (earnings_0Y+earnings_est)/equity           # new roe ratio
pb = bq.data.PX_TO_BOOK_RATIO(fill="prev")        # latest pb ratio

factors = {
    'roe':my_transforms.ZScore(bq, roe),
    'pb' :my_transforms.ZScore(bq, pb),
}
```

<h2 style="color:orange">
    <a name="explore factors"></a>Calculate factors
</h2>

```python
explored_data = context.explore(factors, tab_title='Factors explorer')
```

Here the returned variable `explored_data` is of type `dataexplorer.data_explorer_widget.DataExplorer`.

<h2 style="color:orange">Portfolio Construction</h2>

<h3 style="color:orange">a. Create long-only model portfolio</h3>

We construct <code>long_short_returns</code>, which has a weighting function with the strategy of going long the top quantile, going short the bottom quantile. This weighting function is provided by bqfactor: <code>bqw.top_bottom_quantile</code>.

```python
import bqfactor.reports as bqr
import bqfactor.weighting_functions as bqw

ls_weights = bqr.LongShortPortfolioWeights(weight_function=bqw.top_bottom_quantile)
long_short_returns = bqr.PortfolioReturns(ls_weights, cumulative=False)
```

<h3 style="color:orange">b. Customize weight transformation function</h3>

Below shows how to create a new weighting function which sets weights to 0 if 20D return is negative, and equally weighted the rest of the securities.

```python
from bqfactor import create_rebalancing_config

# declare a default portfolio
default_portfolio = bqr.LongOnlyPortfolioWeights(weight_function=bqw.top_bottom_quantile)

# build a custom  rebalancing config
rebal_config = create_rebalancing_config(bq)

# give config access to market cap data
mkt_cap_data_dict = {'market_cap': bq.data.cur_mkt_cap()}
rebal_config = rebal_config.with_bql_data(mkt_cap_data_dict)

# function to weight the StrategyPortfolio by market cap
def weight_by_mkt_cap(date, target_weights, backtest):
    mkt_cap = backtest.data.get('market_cap')['value']
    prev_weights = target_weights.get_weights(include_cash=False)['weight']
    new_weights = mkt_cap / mkt_cap.sum()
    return target_weights.with_noncash_positions(new_weights)

# assign function to config
rebal_config = rebal_config.with_weight_transformations({'market_cap': weight_by_mkt_cap})

# declare a new StrategyPortfolio
mkt_cap_weighted_port = bqr.StrategyPortfolio(default_portfolio, rebal_config)
```

<h2 style="color: orange;">Construct portfolio using a risk model and optimizer</h2>